### (1.0*) implement Neural Network Matrix Factorization (without Spark):

In [68]:
import numpy as np
import theano
import theano.tensor as T
import lasagne as L
import time

Function for loading dataset:

In [537]:
def load_dataset():
    users, movies, ratings = [], [], []
    
    for line in open('ml-latest-small/ratings.csv').readlines()[1:]:
        user, movie, rating, timestamp = line.split(',')
        users.append(int(user))
        movies.append(int(movie))
        ratings.append(float(rating))
        
    return np.array(users), np.array(movies), np.array(ratings)

In [538]:
users, movies, ratings = load_dataset()

Re-enumerate users and movies:

In [539]:
users = users - 1

count = 0
movieNumbers = {}
for movie in set(movies):
    movieNumbers[movie] = count
    count += 1
    
movies = np.array([movieNumbers[movie] for movie in movies])

Split into train and test parts (put one movie rating into test part for each user):

In [552]:
def train_test_split(users, movies, ratings):
    indices = np.arange(len(ratings))
    np.random.shuffle(indices)
    
    users_train, movies_train, ratings_train = [], [], []
    users_test, movies_test, ratings_test = [], [], []
    users_set = set()
    
    for user, movie, rating in zip(users[indices], movies[indices], ratings[indices]):
        if user in users_set:
            users_train.append(user)
            movies_train.append(movie)
            ratings_train.append(rating)
        else:
            users_set.add(user)
            users_test.append(user)
            movies_test.append(movie)
            ratings_test.append(rating)
        
    return (np.array(users_train), np.array(movies_train), np.array(ratings_train)), (np.array(users_test), np.array(movies_test), np.array(ratings_test))

In [553]:
train_data, test_data = train_test_split(users, movies, ratings)

Model parameters, input and shared variables:

In [585]:
D  = 10
Dp = 60
K  = 1

n    = 1 + max(users)
m    = 1 + max(movies)
size = len(ratings)

U  = theano.shared(np.zeros((n, D)))
Up = theano.shared(np.zeros((n, Dp, K)))
V  = theano.shared(np.zeros((m, D)))
Vp = theano.shared(np.zeros((m, Dp, K)))

U_indices = T.vector("users", dtype="int32")
V_indices = T.vector("movies", dtype="int32")
X         = T.vector("ratings", dtype="float32")

Make set of features for each batch element:

In [586]:
Ux  = U[U_indices]
Vx  = V[V_indices]
Upx = Up[U_indices]
Vpx = Vp[V_indices]

UVp_prods = (Upx * Vpx).sum(axis=2)  # (size, Dp, K), (size, Dp, K) -> (size, Dp)

features = T.concatenate([Ux, Vx, UVp_prods], axis=1)

Neural network:

In [595]:
units = 50
lam  = 0.0 # 1e-4

input_layer = L.layers.InputLayer(shape=[None, 2 * D + Dp], input_var=features)
hidden1     = L.layers.DenseLayer(input_layer, num_units=units, nonlinearity=L.nonlinearities.sigmoid)
hidden2     = L.layers.DenseLayer(hidden1, num_units=units, nonlinearity=L.nonlinearities.sigmoid)
hidden3     = L.layers.DenseLayer(hidden2, num_units=units, nonlinearity=L.nonlinearities.sigmoid)
output      = L.layers.DenseLayer(hidden3, num_units=1, nonlinearity=None)

#preds = L.layers.get_output(output)
preds = features.sum(axis=1)
loss = ((X - preds) ** 2).mean() + lam * ((U ** 2).sum() + (V ** 2).sum() + (Up ** 2).sum() + (Vp ** 2).sum())
rmse = (((X - preds) ** 2).mean()) ** 0.5

Train/predict/etc. functions:

In [599]:
nn_params = L.layers.get_all_params(output, trainable=True)
mf_params = [U, V, Up, Vp]

#train_nn = theano.function([U_indices, V_indices, X], [loss, rmse], updates=L.updates.rmsprop(loss, nn_params, learning_rate=0.001), allow_input_downcast=True)
train_mf = theano.function([U_indices, V_indices, X], [loss, rmse], updates=L.updates.rmsprop(loss, mf_params, learning_rate=0.1), allow_input_downcast=True)
predict  = theano.function([U_indices, V_indices], preds, allow_input_downcast=True)
validate = theano.function([U_indices, V_indices, X], rmse, allow_input_downcast=True)


Helper functions for training:

In [600]:
def iterate_minibatches(users, movies, ratings, batch_size):
    indices = np.arange(len(ratings))
    np.random.shuffle(indices)

    for start_index in range(0, len(ratings) - batch_size + 1, batch_size):
        excerpt = indices[start_index:(start_index + batch_size)]
        #print excerpt
        #print users
        yield users[excerpt], movies[excerpt], ratings[excerpt]

In [601]:
def reset_weights():    
    for v in nn_params:
        val = v.get_value()
        if(len(val.shape) < 2):
            v.set_value(L.init.Constant(0.0)(val.shape))
        else:
            v.set_value(L.init.GlorotUniform(gain=4 * np.sqrt(3))(val.shape))
            
    for v in mf_params:
        v.set_value(np.random.normal(scale=2.0, size=v.get_value().shape))

Main loop:

In [602]:
def run_train(train_data, test_data, num_epochs=200, batch_size=500):
    reset_weights()
    
    users_train, movies_train, ratings_train = train_data
    users_test, movies_test, ratings_test = test_data
    
    for epoch in range(num_epochs):
        train_err = np.array([0.0, 0.0])
        train_batches = 0
        start_time = time.time()
        
        for batch in iterate_minibatches(users_train, movies_train, ratings_train, batch_size):
            users_b, movies_b, ratings_b = batch
            #train_err += train_nn(users_b, movies_b, ratings_b)
            train_err += train_mf(users_b, movies_b, ratings_b)
            train_batches += 2

        val_rmse = float(validate(users_test, movies_test, ratings_test))
        
        print("Epoch {} of {} took {:.3f}s".format(
            epoch + 1, num_epochs, time.time() - start_time))

        print("  train loss (in-iteration):\t\t{:.6f}".format(train_err[0] / train_batches))
        print("  train RMSE (in-iteration):\t\t{:.6f}".format(train_err[1] / train_batches))
        print("  validation RMSE:\t\t\t{:.6f}".format(val_rmse))

In [ ]:
run_train(train_data, test_data)

Epoch 1 of 200 took 1.924s
  train loss (in-iteration):		177.690024
  train RMSE (in-iteration):		8.940956
  validation RMSE:			14.968235
Epoch 2 of 200 took 1.863s
  train loss (in-iteration):		33.536570
  train RMSE (in-iteration):		4.083215
  validation RMSE:			11.806075
Epoch 3 of 200 took 1.913s
  train loss (in-iteration):		25.687782
  train RMSE (in-iteration):		3.566768
  validation RMSE:			9.083315
Epoch 4 of 200 took 2.052s
  train loss (in-iteration):		18.519882
  train RMSE (in-iteration):		3.031097
  validation RMSE:			7.273048
Epoch 5 of 200 took 2.148s
  train loss (in-iteration):		15.785176
  train RMSE (in-iteration):		2.798715
  validation RMSE:			6.445881
Epoch 6 of 200 took 1.855s
  train loss (in-iteration):		13.628073
  train RMSE (in-iteration):		2.602597
  validation RMSE:			5.899555
Epoch 7 of 200 took 2.154s
  train loss (in-iteration):		12.677491
  train RMSE (in-iteration):		2.510096
  validation RMSE:			5.630488
Epoch 8 of 200 took 2.165s
  train loss (in-i

Results are slightly worse than in original article, but maybe that happens because of slightly different model parameters or lack of training epochs. 